# Imports and env set up

In [1]:
import requests
import pandas as pd
from typing import Optional
import os
from dotenv import load_dotenv
import datetime

In [2]:
if load_dotenv():
    API_KEY = os.getenv("open_weather_apikey")
else:
    print('cant find any open weather api key')
    exit()

# Custom functions

In [3]:
def coordinate_retrieve(city_name: str, api_key: str = API_KEY) -> Optional[tuple]:
    url = "https://api.openweathermap.org/geo/1.0/direct"

    params = {"q": city_name,
              "appid": api_key,
              "lang": "en",
              'limit': 1}
    try:
        response = requests.get(url, params=params).json()
        return response[0]['lat'], response[0]['lon']
    except Exception as e:
        return (e.args[0])



In [4]:
def get_current_weather(city_name, api_key: str = API_KEY) -> Optional[
    str]:
    # latitude,longitude = coordinate_retrieve(city_name, api_key)
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric',  # pour obtenir les données en degrés Celsius
        'lang': 'fr'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'temp_min': data['main']['temp_min'],
            'temp_max': data['main']['temp_max'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            'wind_speed': data['wind']['speed'],
            'local_time': str(datetime.datetime.fromtimestamp(data['dt']).astimezone(
                datetime.timezone(datetime.timedelta(seconds=data['timezone']))))
        }
        return weather
    else:
        return None

In [5]:
def weather_over5(city_name, api_key: str = API_KEY) -> Optional[
    pd.DataFrame]:
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric',  # pour obtenir les données en degrés Celsius
        'lang': 'fr'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        result = []
        for item in data['list']:
            weather = {
                'city': data['city']['name'],
                'temperature': item['main']['temp'],
                'temp_min': item['main']['temp_min'],
                'temp_max': item['main']['temp_max'],
                'weather': item['weather'][0]['description'],
                'humidity': item['main']['humidity'],
                'pressure': item['main']['pressure'],
                'wind_speed': item['wind']['speed'],
                'local_time': str(datetime.datetime.fromtimestamp(item['dt']).astimezone(
                    datetime.timezone(datetime.timedelta(seconds=data['city']['timezone']))))
            }
            result.append(weather)
        return pd.DataFrame.from_dict(result)
    else:
        return None

In [6]:
def get_air_pollution_data(city_name: str, api_key: str = API_KEY) -> Optional[
    str]:
    url = "https://api.openweathermap.org/data/2.5/air_pollution"
    latitude, longitude = coordinate_retrieve(city_name)
    params = {
        'lat': latitude,
        'lon': longitude,
        'appid': api_key,
        'lang': 'fr'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['list']
        result = {
            'city': city_name,
            'aqi': data[0]['main']['aqi'],
            'co': data[0]['components']['co'],
            'no': data[0]['components']['no'],
            'no2': data[0]['components']['no2'],
            'o3': data[0]['components']['o3'],
            'so2': data[0]['components']['so2'],
            'pm2_5': data[0]['components']['pm2_5'],
            'pm10': data[0]['components']['pm10'],
            'nh3': data[0]['components']['nh3'],
            'local_time': str(datetime.datetime.fromtimestamp(data[0]['dt']))
        }
        return result
    else:
        return None

In [7]:
def air_pollution_data_over5(city_name: str, start: int, api_key: str = API_KEY) -> Optional[
    pd.DataFrame]:
    url = "https://api.openweathermap.org/data/2.5/air_pollution/history"
    latitude, longitude = coordinate_retrieve(city_name)
    end = int((datetime.datetime.fromtimestamp(start) + datetime.timedelta(days=4)).timestamp())
    params = {
        'lat': latitude,
        'lon': longitude,
        'start': start,
        'end': end,
        'appid': api_key,
    }
    response = requests.get(url, params=params)
    # print(response.status_code)
    if response.status_code == 200:
        results = []
        data = response.json()['list']
        for item in data:
            result = {
                'city': city_name,
                'aqi': item['main']['aqi'],
                'co': item['components']['co'],
                'no': item['components']['no'],
                'no2': item['components']['no2'],
                'o3': item['components']['o3'],
                'so2': item['components']['so2'],
                'pm2_5': item['components']['pm2_5'],
                'pm10': item['components']['pm10'],
                'nh3': item['components']['nh3'],
                'local_time': str(datetime.datetime.fromtimestamp(item['dt']))
            }
            results.append(result)
        return pd.DataFrame.from_records(results)
    else:
        return None


# API CAll examples

## Single weather data API call

In [8]:
weather = get_current_weather('Paris')
if weather:
    print(f"Weather in {weather['city']}:")
    print(f"Temperature: {weather['temperature']}°C")
    print(f"Description: {weather['description']}")
    print(f"Humidity: {weather['humidity']}%")
    print(f"Pressure: {weather['pressure']} hPa")
    print(f"Wind Speed: {weather['wind_speed']} m/s")
else:
    print("Failed to get weather data")

Weather in Paris:
Temperature: 22.13°C
Description: nuageux
Humidity: 54%
Pressure: 1015 hPa
Wind Speed: 8.23 m/s


## 5 day / 3 hours weather forecast

In [9]:
weather_over5('Paris')

,city,temperature,temp_min,temp_max,weather,humidity,pressure,wind_speed,local_time
0,Paris,22.18,22.18,22.53,légère pluie,52,1015,5.25,2024-07-16 17:00:00+02:00
1,Paris,21.76,21.63,21.76,légère pluie,56,1016,3.39,2024-07-16 20:00:00+02:00
2,Paris,18.89,18.89,18.89,couvert,68,1018,1.67,2024-07-16 23:00:00+02:00
3,Paris,17.88,17.88,17.88,couvert,70,1019,1.91,2024-07-17 02:00:00+02:00
4,Paris,15.93,15.93,15.93,nuageux,77,1020,1.54,2024-07-17 05:00:00+02:00
5,Paris,16.95,16.95,16.95,partiellement nuageux,73,1021,1.23,2024-07-17 08:00:00+02:00
6,Paris,22.28,22.28,22.28,ciel dégagé,51,1021,0.77,2024-07-17 11:00:00+02:00
7,Paris,26.66,26.66,26.66,ciel dégagé,35,1021,0.77,2024-07-17 14:00:00+02:00
8,Paris,27.80,27.80,27.80,ciel dégagé,31,1020,0.61,2024-07-17 17:00:00+02:00
9,Paris,25.72,25.72,25.72,peu nuageux,38,1020,1.95,2024-07-17 20:00:00+02:00


In [10]:
weather_over5('Shanghai')

,city,temperature,temp_min,temp_max,weather,humidity,pressure,wind_speed,local_time
0,Shanghai,30.28,27.95,30.28,partiellement nuageux,82,1008,4.75,2024-07-16 23:00:00+08:00
1,Shanghai,28.60,27.18,28.60,nuageux,83,1008,4.40,2024-07-17 02:00:00+08:00
2,Shanghai,26.41,26.41,26.41,nuageux,85,1007,3.91,2024-07-17 05:00:00+08:00
3,Shanghai,29.29,29.29,29.29,nuageux,72,1007,4.11,2024-07-17 08:00:00+08:00
4,Shanghai,33.80,33.80,33.80,couvert,53,1007,4.03,2024-07-17 11:00:00+08:00
5,Shanghai,36.48,36.48,36.48,couvert,45,1006,3.46,2024-07-17 14:00:00+08:00
6,Shanghai,36.08,36.08,36.08,partiellement nuageux,46,1005,2.41,2024-07-17 17:00:00+08:00
7,Shanghai,29.92,29.92,29.92,nuageux,74,1006,2.79,2024-07-17 20:00:00+08:00
8,Shanghai,28.93,28.93,28.93,couvert,77,1005,2.60,2024-07-17 23:00:00+08:00
9,Shanghai,28.14,28.14,28.14,couvert,80,1004,2.48,2024-07-18 02:00:00+08:00


In [11]:
weather_over5('Toronto')

,city,temperature,temp_min,temp_max,weather,humidity,pressure,wind_speed,local_time
0,Toronto,21.02,20.17,21.02,pluie modérée,94,1005,2.42,2024-07-16 11:00:00-04:00
1,Toronto,23.94,23.94,25.19,pluie modérée,77,1006,1.10,2024-07-16 14:00:00-04:00
2,Toronto,27.04,27.04,27.04,légère pluie,66,1006,3.88,2024-07-16 17:00:00-04:00
3,Toronto,22.72,22.72,22.72,légère pluie,91,1007,3.31,2024-07-16 20:00:00-04:00
4,Toronto,21.01,21.01,21.01,légère pluie,97,1007,1.86,2024-07-16 23:00:00-04:00
5,Toronto,20.05,20.05,20.05,légère pluie,97,1008,2.18,2024-07-17 02:00:00-04:00
6,Toronto,19.14,19.14,19.14,ciel dégagé,95,1007,1.80,2024-07-17 05:00:00-04:00
7,Toronto,19.60,19.60,19.60,peu nuageux,83,1009,1.71,2024-07-17 08:00:00-04:00
8,Toronto,24.27,24.27,24.27,ciel dégagé,55,1009,2.38,2024-07-17 11:00:00-04:00
9,Toronto,24.69,24.69,24.69,légère pluie,59,1009,4.24,2024-07-17 14:00:00-04:00


## Single Air Pollution data api call

In [12]:
get_air_pollution_data('Paris')

{'city': 'Paris',
 'aqi': 2,
 'co': 158.55,
 'no': 0.35,
 'no2': 1.41,
 'o3': 62.94,
 'so2': 0.5,
 'pm2_5': 0.56,
 'pm10': 0.79,
 'nh3': 0.57,
 'local_time': '2024-07-16 14:12:44'}

## Forecast for 4 days ahead with 1-hour step

In [13]:
stamp = int(datetime.datetime.now().timestamp())
air_pollution_data_over5('Paris', stamp)

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Paris,2,160.22,0.30,1.22,65.09,0.46,0.50,0.70,0.55,2024-07-16 15:00:00
1,Paris,2,160.22,0.28,1.30,65.80,0.45,0.50,0.68,0.56,2024-07-16 16:00:00
2,Paris,2,161.89,0.28,1.59,65.80,0.45,0.50,0.67,0.63,2024-07-16 17:00:00
3,Paris,2,161.89,0.24,1.86,65.80,0.49,0.50,0.67,0.74,2024-07-16 18:00:00
4,Paris,2,163.56,0.17,2.10,65.80,0.57,0.52,0.72,0.89,2024-07-16 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
91,Paris,1,181.91,2.26,6.17,41.49,0.94,4.17,5.80,2.09,2024-07-20 10:00:00
92,Paris,1,181.91,1.76,4.67,50.07,0.97,3.62,5.05,2.22,2024-07-20 11:00:00
93,Paris,2,183.58,0.56,2.96,68.67,0.75,2.94,3.98,1.69,2024-07-20 12:00:00
94,Paris,2,181.91,0.37,2.25,76.53,0.69,3.12,4.07,1.57,2024-07-20 13:00:00


In [14]:
air_pollution_data_over5('Shanghai', stamp)

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Shanghai,5,2056.12,107.29,61.69,0.00,350.95,49.04,72.62,0.0,2024-07-16 15:00:00
1,Shanghai,5,2082.82,109.08,56.21,0.00,350.95,48.25,70.84,0.0,2024-07-16 16:00:00
2,Shanghai,5,2216.34,119.81,55.52,0.00,377.66,51.70,74.60,0.0,2024-07-16 17:00:00
3,Shanghai,5,2376.56,128.75,56.89,0.00,404.36,54.01,76.73,0.0,2024-07-16 18:00:00
4,Shanghai,5,2269.75,121.59,57.58,0.00,392.91,51.77,73.12,0.0,2024-07-16 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
91,Shanghai,5,3952.03,153.78,167.25,9.48,625.61,111.52,139.69,0.0,2024-07-20 10:00:00
92,Shanghai,5,4753.11,219.94,164.51,4.02,694.28,135.42,169.10,0.0,2024-07-20 11:00:00
93,Shanghai,5,6034.85,318.29,169.99,1.16,862.12,179.54,223.47,0.0,2024-07-20 12:00:00
94,Shanghai,5,8117.68,457.76,183.70,0.07,1159.67,255.17,308.03,0.0,2024-07-20 13:00:00


In [15]:
air_pollution_data_over5('Toronto', stamp)

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Toronto,2,373.84,0.18,29.47,29.68,13.11,14.59,16.05,1.44,2024-07-16 15:00:00
1,Toronto,2,370.50,0.40,29.13,22.17,12.52,12.97,14.08,1.36,2024-07-16 16:00:00
2,Toronto,2,357.15,1.15,26.05,21.28,11.68,10.37,11.29,1.25,2024-07-16 17:00:00
3,Toronto,1,330.45,2.18,21.25,27.54,10.49,7.55,8.28,1.06,2024-07-16 18:00:00
4,Toronto,1,317.10,2.46,18.34,36.84,9.78,5.89,6.55,1.00,2024-07-16 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
91,Toronto,1,253.68,0.05,10.11,20.74,2.15,4.00,5.12,0.61,2024-07-20 10:00:00
92,Toronto,1,250.34,0.13,10.54,15.74,2.27,4.23,5.42,0.52,2024-07-20 11:00:00
93,Toronto,1,260.35,0.37,11.82,10.73,2.47,4.40,5.81,0.55,2024-07-20 12:00:00
94,Toronto,1,317.10,2.63,16.11,5.99,3.37,4.62,6.55,1.01,2024-07-20 13:00:00
